# Métodos numéricos para resolver el problema de desintegración radiactiva

Un material radiactivo se desintegra a una razón proporcional a la cantidad presente. Si inicialmente hay 60 mg de material y al cabo de una hora se observa que ha perdido el 7\% de la cantidad inicial, hallar:

1. El material de masa en cualquier momento $t$.

2. La masa del material después de 4 horas.

3. El tiempo que transcurre hasta la desintegración de la mitad de la cantidad inicial. 

Se presentan los siguientes métodos:
* Euler
* Euler modificado
* Runge-Kutta de orden 4

#### La siguiente celda solo se debe ejecutar la primera vez que se use el notebook para instalar las bibliotecas necesarias.

In [ ]:
# Instalar las bibliotecas
%pip install plotly --q
%pip install nbformat --q
%pip install sympy --q

In [2]:
# Importar las bibliotecas
from sympy import lambdify, sympify # ¡¡¡NO MODIFICAR!!!
from sympy.abc import t, n # ¡¡¡NO MODIFICAR!!!
import plotly.express as px # ¡¡¡NO MODIFICAR!!!
import plotly.graph_objects as go # ¡¡¡NO MODIFICAR!!!
import numpy as np # ¡¡¡NO MODIFICAR!!!

### Definición de constantes y funciones

In [3]:
t_0 = 0 # Tiempo inicial en minutos
t_f = 15 # Tiempo final en minutos
valores_t = np.linspace(t_0, t_f, (t_f-t_0)*15) # Vector de tiempo !!!NO MODIFICAR!!!
t_n = [4] # Valores en los que se quiere conocer la solucion en minutos
porcentajes = [0.5] # Porcentajes de desintegración

k = 0.07257  # contante de desintegración ????
C = 60 # Constante de integración

f_real = f'-{k}*n' # Ecuacion diferencial de la forma dy/dx = f(x, y) ¡¡¡NO MODIFICAR!!!
f_real = lambdify(n, sympify(f_real), 'numpy') # Funcion f(x, y) en formato numpy ¡¡¡NO MODIFICAR!!!
f_sol_analitica = f'{C}*exp(-{k}*t)' # Solucion analitica de la ecuacion diferencial ¡¡¡NO MODIFICAR!!!
f_sol_analitica = lambdify(t, sympify(f_sol_analitica), 'numpy') # Funcion f(x, y) en formato numpy ¡¡¡NO MODIFICAR!!!

### Analitica

In [4]:
def solucion_analitica(f, valores_t, C, k):
    # Evaluar la función
    valores_y = list(map(lambda t: f(t), valores_t)) # Evaluar la función en el vector de tiempo

    # Graficar
    titulo = f'Desintegración de material radioactivo<br>Solucion analítica: N(t)={round(C,3)}exp(-{round(k,3)}t)' # Titulo de la gráfica
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'N(t)[mg]' # Nombre del eje y 

    fig = go.Figure() # Crear figura para graficar
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # Actualizar diseño de la gráfica
    plot_analitica = go.Scatter(x=valores_t, y=valores_y, name='Solución analítica', line=dict(color='blue', width=2), mode='lines', showlegend=True) # Crear gráfica de la solución analítica
    fig.add_trace(plot_analitica) # Agregar gráfica de la solución analítica a la figura
    fig.show() # Mostrar figura
    return plot_analitica # Regresar gráfica de la solución analítica

plot_Analitica = solucion_analitica(f_sol_analitica, valores_t, C, k) # Graficar solución analítica

### Graficar masa en un tiempo t y el tiempo que transcurre hasta alcanzar un porcentaje de desintegración

In [6]:
def marcadores(f, k, t_n, porcentajes):
    # Evaluar la función en los valores de tiempo
    valores_y = tuple(map(lambda t: f(t), t_n)) # Convert the list to a tuple

    # Encontrar el tiempo en el que se alcanza el porcentaje de desintegración
    porcentajes.sort(reverse=True) # Ordenar los porcentajes de mayor a menor
    t_porcentajes = tuple(map(lambda porcentaje: -np.log(porcentaje)/k, porcentajes)) # Convert the list to a tuple
    y_porcentajes = tuple(map(lambda t: f(t), t_porcentajes)) # Convert the list to a tuple

    # Graficar
    titulo = f'Desintegración de material radioactivo<br>Porcentaje(s) de desintegración: {[round(p*100,3) for p in porcentajes]}%<br>Evaluación en t: {[round(t_,3) for t_ in t_n]}' # Titulo de la gráfica
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'N(t)[mg]' # Nombre del eje y 
    fig = go.Figure() # Crear figura para graficar
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # Actualizar diseño de la gráfica
    fig.add_trace(plot_Analitica) # Agregar gráfica de la solución analítica a la figura
    plot_tiempos = go.Scatter(x=t_n, y=valores_y, name=f'Funcion evaluada en t={t_n}', marker=dict(color='red', size=9), mode='markers', showlegend=True) # Graficar los valores de tiempo
    fig.add_trace(plot_tiempos) # Agregar gráfica de los valores de tiempo
    plot_porcentajes = go.Scatter(x=t_porcentajes, y=y_porcentajes, name=f'Porcentajes de desintegración<br>{[p*100 for p in porcentajes]}', marker=dict(color='orange', size=9), mode='markers', showlegend=True) # Graficar el porcentaje de desintegración
    fig.add_trace(plot_porcentajes) # Agregar gráfica de los porcentajes de desintegración
    fig.show() # Mostrar figura
    return plot_tiempos, plot_porcentajes # Regresar gráfica de los valores de tiempo y porcentaje de desintegración

plot_Tiempos, plot_Porcentajes = marcadores(f_sol_analitica, k, t_n, porcentajes) # Graficar marcadores

### Definición de constante para métodos numéricos

In [7]:
h = 0.2 # Tamaño de paso
n_pasos = int(np.ceil((t_f-t_0)/h)) # Número de pasos !!!NO MODIFICAR!!!
y_0 = C # Masa inicial !!!NO MODIFICAR!!!

### Euler

In [8]:
def metodo_euler(f, t_0, y_0, h, n_pasos):
    valores_y_Euler = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo
    
    # Iterar sobre el número de pasos y aplicar el método de Euler
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # Agregar el valor de t a la lista
        y_nueva = valores_y_Euler[i] + h * f(valores_y_Euler[i]) # Método de Euler
        valores_y_Euler.append(y_nueva) # Agregar el valor de y a la lista

    # Graficar
    titulo = f'Desintegración de material radioactivo<br>Método de Euler con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'N(t)[mg]' # Nombre del eje y

    fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # ¡¡¡NO MODIFICAR!!!
    plot_Euler = go.Scatter(x=valores_t, y=valores_y_Euler, mode='markers', marker=dict(size=8, symbol='diamond'), name='Euler', marker_color='lime', showlegend=True) # ¡¡¡NO MODIFICAR!!!
    fig.add_trace(plot_Euler) # ¡¡¡NO MODIFICAR!!!
    fig.show() # ¡¡¡NO MODIFICAR!!!
    return plot_Euler # ¡¡¡NO MODIFICAR!!!

plot_Euler = metodo_euler(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler

### Euler modificado

In [9]:
def metodo_euler_mod(f, t_0, y_0, h, n_pasos):
    valores_y_Euler_Mod = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Euler modificado
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h)
        y_nueva = valores_y_Euler_Mod[i] + h * f(valores_y_Euler_Mod[i] + h/2 * f(valores_y_Euler_Mod[i])) # ¡¡¡NO MODIFICAR!!!
        valores_y_Euler_Mod.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Desintegración de material radioactivo<br>Método de Euler modificado con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'N(t)[mg]' # Nombre del eje y

    fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # ¡¡¡NO MODIFICAR!!!
    plot_Euler_Mod = go.Scatter(x=valores_t, y=valores_y_Euler_Mod, mode='markers', marker=dict(size=8, symbol='triangle-up'), name='Euler modificado', marker_color='darkviolet', showlegend=True) # ¡¡¡NO MODIFICAR!!!
    fig.add_trace(plot_Euler_Mod) # ¡¡¡NO MODIFICAR!!!
    fig.show() # ¡¡¡NO MODIFICAR!!!
    return plot_Euler_Mod # ¡¡¡NO MODIFICAR!!!

plot_Euler_Mod = metodo_euler_mod(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler modificado

## Runge-Kutta de orden 4

In [10]:
def metodo_RK_4(f, t_0, y_0, h, n_pasos):
    valores_y_RK_4 = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Runge-Kutta 4
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # ¡¡¡NO MODIFICAR!!!
        k1 = h * f(valores_y_RK_4[i]) # ¡¡¡NO MODIFICAR!!!
        k2 = h * f(valores_y_RK_4[i] + k1/2) # ¡¡¡NO MODIFICAR!!!
        k3 = h * f(valores_y_RK_4[i] + k2/2) # ¡¡¡NO MODIFICAR!!!
        k4 = h * f(valores_y_RK_4[i] + k3) # ¡¡¡NO MODIFICAR!!!
        y_nueva = valores_y_RK_4[i] + 1/6 * (k1 + 2*k2 + 2*k3 + k4) # ¡¡¡NO MODIFICAR!!!
        valores_y_RK_4.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Desintegración de material radioactivo<br>Método de Runge-Kutta 4 con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'N(t)[mg]' # Nombre del eje y

    fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
    fig.update_layout(title=titulo, title_x=0.5, xaxis_title=eje_x, yaxis_title=eje_y) # ¡¡¡NO MODIFICAR!!!
    plot_RK_4 = go.Scatter(x=valores_t, y=valores_y_RK_4, mode='markers', marker=dict(size=8, symbol='triangle-down'), name='Runge-Kutta 4', marker_color='hotpink', showlegend=True) # ¡¡¡NO MODIFICAR!!!
    fig.add_trace(plot_RK_4) # ¡¡¡NO MODIFICAR!!!
    fig.show() # ¡¡¡NO MODIFICAR!!!
    return plot_RK_4 # ¡¡¡NO MODIFICAR!!!

plot_RK_4 = metodo_RK_4(f_real, t_0, y_0, h, n_pasos) # Graficar método de Runge-Kutta 4

## Comparacion de los metodos

In [11]:
titulo = f'Desintegración de material radioactivo<br>Comparación de metodos numericos y solucion analitica<br>Porcentaje(s) de desintegración: {[round(p*100,3) for p in porcentajes]}%<br>Evaluación en t: {[round(t_,3) for t_ in t_n]}' # ¡¡¡NO MODIFICAR!!!
eje_x = 't[min]' # Nombre del eje x
eje_y = 'N(t)[mg]' # Nombre del eje y

fig = go.Figure() # ¡¡¡NO MODIFICAR!!!
fig.update_layout(title=titulo, xaxis_title=eje_x, yaxis_title=eje_y, legend_title='Métodos', title_x=0.5, title_y=0.9) # ¡¡¡NO MODIFICAR!!!
fig.update_layout(margin=dict(t=130)) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Analitica) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Euler) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Euler_Mod) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_RK_4) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Tiempos) # ¡¡¡NO MODIFICAR!!!
fig.add_trace(plot_Porcentajes) # ¡¡¡NO MODIFICAR!!!

fig.show() # ¡¡¡NO MODIFICAR!!!